In [4]:
import os
import time
import torch
import torchtuples as tt
import numpy as np
import easydict
import pandas as pd

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import StratifiedKFold


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import StratifiedKFold


In [6]:
import os
import time
import torch
import torchtuples as tt
import numpy as np
import easydict
import pandas as pd

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import StratifiedKFold

## Data Preprocessing

In [7]:
# from google.colab import files
# uploaded = files.upload()

In [8]:
df = pd.read_csv('Dataset/processdata.csv', encoding='latin-1')
data = df

In [9]:
date_columns = ['Date.of.Last.Contact', 'Date.of.Diagnostic']
data[date_columns] = data[date_columns].apply(pd.to_datetime, errors='coerce')
has_na = data[date_columns].isna().any(axis=1)

# My comment: why not death time - contact time?
if has_na.any():
#     max_last_contact_date = data['Date.of.Last.Contact'].max()
#     data['Survival_Time'] = (max_last_contact_date - data['Date.of.Diagnostic']).dt.days
    data['Survival_Time'] = (data['Date.of.Last.Contact'] - data['Date.of.Diagnostic']).dt.days
else:
    data['Survival_Time'] = (data['Date.of.Last.Contact'] - data['Date.of.Diagnostic']).dt.days

In [10]:
data['indicater'] = np.where(data['Date.of.Death'].isna(), 0, 1)
columns_to_drop = ['Date.of.Death', 'Date.of.Last.Contact', 'Date.of.Diagnostic']


data.drop(columns=columns_to_drop, inplace=True)

In [11]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, MinMaxScaler


columns_to_one_hot = ['RCBP.Name', 'Raca.Color', 'State.Civil', 'Code.Profession', 'Name.Occupation', 'Status.Address',
                      'City.Address', 'Description.of.Topography', 'Topography.Code', 'Morphology.Description',
                      'Code.of.Morphology', 'Description.of.Disease', 'Illness.Code', 'Diagnostic.means', 'Extension',
                      'Type.of.Death']

# Replace other values that are not in top 9, into "other"
for column in columns_to_one_hot:
    top_9_values = data[column].value_counts().nlargest(9).index
    data[column] = data[column].where(data[column].isin(top_9_values), 'other')

data = pd.get_dummies(data, columns=columns_to_one_hot)

columns_to_binarize = ['Gender', 'Indicator.of.Rare.Case']

lb = LabelBinarizer()
for column in columns_to_binarize:
    data[column] = lb.fit_transform(data[column])

scaler = MinMaxScaler()
data['Age'] = scaler.fit_transform(data[['Age']])


print(data.head())


   Gender       Age  Indicator.of.Rare.Case  Survival_Time  indicater  \
0       1  0.283019                       0              0          1   
1       1  0.490566                       0              0          1   
2       1  0.698113                       0              0          1   
3       1  0.556604                       0              0          1   
4       0  0.122642                       0           3959          0   

   RCBP.Name_RCBP BELO HORIZONTE  RCBP.Name_RCBP CAMPINAS-UNICAMP  \
0                          False                             True   
1                          False                             True   
2                          False                             True   
3                          False                             True   
4                          False                             True   

   RCBP.Name_RCBP DISTRITO FEDERAL  RCBP.Name_RCBP DRS BARRETOS  \
0                            False                        False   
1           

In [12]:
data.head()

,Gender,Age,Indicator.of.Rare.Case,Survival_Time,indicater,RCBP.Name_RCBP BELO HORIZONTE,RCBP.Name_RCBP CAMPINAS-UNICAMP,RCBP.Name_RCBP DISTRITO FEDERAL,RCBP.Name_RCBP DRS BARRETOS,RCBP.Name_RCBP FORTALEZA,...,Diagnostic.means_PESQUISA,Diagnostic.means_SDO,Diagnostic.means_other,Extension_IN SITU,Extension_LOCALIZADO,Extension_MET?STASE,Extension_N?O SE APLICA,Type.of.Death_C?NCER,Type.of.Death_N?O C?NCER,Type.of.Death_other
0,1,0.283019,0,0,1,False,True,False,False,False,...,False,True,False,False,False,True,False,True,False,False
1,1,0.490566,0,0,1,False,True,False,False,False,...,False,True,False,False,False,True,False,True,False,False
2,1,0.698113,0,0,1,False,True,False,False,False,...,False,True,False,False,False,False,True,True,False,False
3,1,0.556604,0,0,1,False,True,False,False,False,...,False,True,False,False,False,True,False,True,False,False
4,0,0.122642,0,3959,0,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,True


In [13]:
print(list(data.columns))

['Gender', 'Age', 'Indicator.of.Rare.Case', 'Survival_Time', 'indicater', 'RCBP.Name_RCBP BELO HORIZONTE', 'RCBP.Name_RCBP CAMPINAS-UNICAMP', 'RCBP.Name_RCBP DISTRITO FEDERAL', 'RCBP.Name_RCBP DRS BARRETOS', 'RCBP.Name_RCBP FORTALEZA', 'RCBP.Name_RCBP MANAUS', 'RCBP.Name_RCBP MATO GROSSO (INTERIOR)', 'RCBP.Name_RCBP NATAL', 'RCBP.Name_RCBP PORTO ALEGRE', 'RCBP.Name_other', 'Raca.Color_AMARELA', 'Raca.Color_BRANCO', 'Raca.Color_IND?GENA', 'Raca.Color_PARDA', 'Raca.Color_PRETA', 'Raca.Color_other', 'State.Civil_CASADO', 'State.Civil_SEPARADO JUDICIALMENTE', 'State.Civil_SOLTEIRO', 'State.Civil_UNI?O CONSENSUAL', 'State.Civil_VI?VO', 'State.Civil_other', 'Code.Profession_0.0', 'Code.Profession_111.0', 'Code.Profession_141.0', 'Code.Profession_512.0', 'Code.Profession_514.0', 'Code.Profession_516.0', 'Code.Profession_622.0', 'Code.Profession_715.0', 'Code.Profession_782.0', 'Code.Profession_other', 'Name.Occupation_CONDUTORES DE VE?CULOS E OPERADORES DE EQUIPAMENTOS DE ELEVA??O E DE MOVIME

In [14]:
# change the batch size from 1024 to 3222 to improve the performance
# created by Xinyu modified by Jingyan
args = easydict.EasyDict({
    "batch_size": 2048,
    "cuda": True, # should set it to be true when using gpu, otherwise data would be on two devices
    "lr": 0.05,
    "seed": 1111,
    "reduce_rate": 0.95,
    "epochs": 200,
    "clip": 5.0,
    "log_interval":10,
})

In [15]:
def cycle(iterable):
    while True:
        for x in iterable:
            yield x

In [16]:
device = torch.device("cuda" if args.cuda else "cpu")
dtype = torch.FloatTensor
if torch.cuda.is_available() and args.cuda:
    dtype = torch.cuda.FloatTensor

np.random.seed(1234)
_ = torch.manual_seed(123)

In [17]:
print(device)

cuda


In [18]:
from sklearn import preprocessing

In [19]:
data[data['indicater']==1]

,Gender,Age,Indicator.of.Rare.Case,Survival_Time,indicater,RCBP.Name_RCBP BELO HORIZONTE,RCBP.Name_RCBP CAMPINAS-UNICAMP,RCBP.Name_RCBP DISTRITO FEDERAL,RCBP.Name_RCBP DRS BARRETOS,RCBP.Name_RCBP FORTALEZA,...,Diagnostic.means_PESQUISA,Diagnostic.means_SDO,Diagnostic.means_other,Extension_IN SITU,Extension_LOCALIZADO,Extension_MET?STASE,Extension_N?O SE APLICA,Type.of.Death_C?NCER,Type.of.Death_N?O C?NCER,Type.of.Death_other
0,1,0.283019,0,0,1,False,True,False,False,False,...,False,True,False,False,False,True,False,True,False,False
1,1,0.490566,0,0,1,False,True,False,False,False,...,False,True,False,False,False,True,False,True,False,False
2,1,0.698113,0,0,1,False,True,False,False,False,...,False,True,False,False,False,False,True,True,False,False
3,1,0.556604,0,0,1,False,True,False,False,False,...,False,True,False,False,False,True,False,True,False,False
5,0,0.754717,0,0,1,False,True,False,False,False,...,False,True,False,False,False,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14750,0,0.613208,0,1475,1,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False
14751,1,0.528302,0,1302,1,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False
14752,1,0.122642,0,936,1,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False
14753,1,0.773585,0,420,1,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False


In [20]:
X = data.drop(['Survival_Time', 'indicater'], axis=1)
time_all = data['Survival_Time'].values
event_all = data['indicater'].values

In [21]:
# X_normalize
X.head(5)

,Gender,Age,Indicator.of.Rare.Case,RCBP.Name_RCBP BELO HORIZONTE,RCBP.Name_RCBP CAMPINAS-UNICAMP,RCBP.Name_RCBP DISTRITO FEDERAL,RCBP.Name_RCBP DRS BARRETOS,RCBP.Name_RCBP FORTALEZA,RCBP.Name_RCBP MANAUS,RCBP.Name_RCBP MATO GROSSO (INTERIOR),...,Diagnostic.means_PESQUISA,Diagnostic.means_SDO,Diagnostic.means_other,Extension_IN SITU,Extension_LOCALIZADO,Extension_MET?STASE,Extension_N?O SE APLICA,Type.of.Death_C?NCER,Type.of.Death_N?O C?NCER,Type.of.Death_other
0,1,0.283019,0,False,True,False,False,False,False,False,...,False,True,False,False,False,True,False,True,False,False
1,1,0.490566,0,False,True,False,False,False,False,False,...,False,True,False,False,False,True,False,True,False,False
2,1,0.698113,0,False,True,False,False,False,False,False,...,False,True,False,False,False,False,True,True,False,False
3,1,0.556604,0,False,True,False,False,False,False,False,...,False,True,False,False,False,True,False,True,False,False
4,0,0.122642,0,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True


## My comment: We have survival_time = -1 in our data!

In [22]:
print(np.unique(time_all))

[  -1    0    1 ... 7362 7381 7408]


In [23]:
max_time = data['Survival_Time'].max()
max_time

7408

In [24]:
print(list(X.columns))

['Gender', 'Age', 'Indicator.of.Rare.Case', 'RCBP.Name_RCBP BELO HORIZONTE', 'RCBP.Name_RCBP CAMPINAS-UNICAMP', 'RCBP.Name_RCBP DISTRITO FEDERAL', 'RCBP.Name_RCBP DRS BARRETOS', 'RCBP.Name_RCBP FORTALEZA', 'RCBP.Name_RCBP MANAUS', 'RCBP.Name_RCBP MATO GROSSO (INTERIOR)', 'RCBP.Name_RCBP NATAL', 'RCBP.Name_RCBP PORTO ALEGRE', 'RCBP.Name_other', 'Raca.Color_AMARELA', 'Raca.Color_BRANCO', 'Raca.Color_IND?GENA', 'Raca.Color_PARDA', 'Raca.Color_PRETA', 'Raca.Color_other', 'State.Civil_CASADO', 'State.Civil_SEPARADO JUDICIALMENTE', 'State.Civil_SOLTEIRO', 'State.Civil_UNI?O CONSENSUAL', 'State.Civil_VI?VO', 'State.Civil_other', 'Code.Profession_0.0', 'Code.Profession_111.0', 'Code.Profession_141.0', 'Code.Profession_512.0', 'Code.Profession_514.0', 'Code.Profession_516.0', 'Code.Profession_622.0', 'Code.Profession_715.0', 'Code.Profession_782.0', 'Code.Profession_other', 'Name.Occupation_CONDUTORES DE VE?CULOS E OPERADORES DE EQUIPAMENTOS DE ELEVA??O E DE MOVIMENTA??O DE CARGAS   \t', 'Name.

In [25]:
check = pd.DataFrame(columns=['Gender', 'Age', 'Indicator.of.Rare.Case', 'Survival_Time', 'indicater', 'RCBP.Name_RCBP BELO HORIZONTE', 'RCBP.Name_RCBP CAMPINAS-UNICAMP', 'RCBP.Name_RCBP DISTRITO FEDERAL', 'RCBP.Name_RCBP DRS BARRETOS', 'RCBP.Name_RCBP FORTALEZA', 'RCBP.Name_RCBP MANAUS', 'RCBP.Name_RCBP MATO GROSSO (INTERIOR)', 'RCBP.Name_RCBP NATAL', 'RCBP.Name_RCBP PORTO ALEGRE', 'RCBP.Name_other', 'Raca.Color_AMARELA', 'Raca.Color_BRANCO', 'Raca.Color_IND?GENA', 'Raca.Color_PARDA', 'Raca.Color_PRETA', 'Raca.Color_other', 'State.Civil_CASADO', 'State.Civil_SEPARADO JUDICIALMENTE', 'State.Civil_SOLTEIRO', 'State.Civil_UNI?O CONSENSUAL', 'State.Civil_VI?VO', 'State.Civil_other', 'Code.Profession_0.0', 'Code.Profession_111.0', 'Code.Profession_141.0', 'Code.Profession_512.0', 'Code.Profession_514.0', 'Code.Profession_516.0', 'Code.Profession_622.0', 'Code.Profession_715.0', 'Code.Profession_782.0', 'Code.Profession_other', 'Name.Occupation_CONDUTORES DE VE?CULOS E OPERADORES DE EQUIPAMENTOS DE ELEVA??O E DE MOVIMENTA??O DE CARGAS   \t', 'Name.Occupation_GERENTES DE PRODU??O E OPERA??ES  \t', 'Name.Occupation_MEMBROS SUPERIORES DO PODER LEGISLATIVO, EXECUTIVO E JUDICI?RIO   \t', 'Name.Occupation_SEM INFORMA??O\t', 'Name.Occupation_TRABALHADORES AGR?COLAS   \t', 'Name.Occupation_TRABALHADORES DA CONSTRU??O CIVIL E OBRAS P?BLICAS\t', 'Name.Occupation_TRABALHADORES DOS SERVI?OS DOM?STICOS EM GERAL\t', 'Name.Occupation_TRABALHADORES NOS SERVI?OS DE ADMINISTRA??O, CONSERVA??O E MANUTEN??O DE EDIF?CIOS E LOGRADOUROS  \t', 'Name.Occupation_TRABALHADORES NOS SERVI?OS DE EMBELEZAMENTO E CUIDADOS PESSOAIS   \t', 'Name.Occupation_other', 'Status.Address_AMAZONAS', 'Status.Address_CEAR?', 'Status.Address_DISTRITO FEDERAL', 'Status.Address_MATO GROSSO ', 'Status.Address_MINAS GERAIS', 'Status.Address_RIO DE JANEIRO', 'Status.Address_RIO GRANDE DO NORTE', 'Status.Address_RIO GRANDE DO SUL', 'Status.Address_S?O PAULO', 'Status.Address_other', 'City.Address_BARRETOS', 'City.Address_BEBEDOURO', 'City.Address_BRAS?LIA', 'City.Address_CAMPINAS', 'City.Address_COLINA', 'City.Address_GUA?RA', 'City.Address_MONTE AZUL PAULISTA', 'City.Address_OL?MPIA', 'City.Address_VIRADOURO', 'City.Address_other', 'Description.of.Topography_COLO DO UTERO, SOE', 'Description.of.Topography_ESTOMAGO, SOE', 'Description.of.Topography_LOCALIZACAO PRIMARIA DESCONHECIDA, SOE', 'Description.of.Topography_MAMA , SOE', 'Description.of.Topography_MEDULA OSSEA', 'Description.of.Topography_PELE DE OUTRAS PARTES DA FACE', 'Description.of.Topography_PELE DO MEMBRO SUPERIOR E DO OMBRO', 'Description.of.Topography_PROSTATA, SOE', 'Description.of.Topography_PULMAO, SOE', 'Description.of.Topography_other', 'Topography.Code_C169', 'Topography.Code_C349', 'Topography.Code_C421', 'Topography.Code_C443', 'Topography.Code_C446', 'Topography.Code_C509', 'Topography.Code_C539', 'Topography.Code_C619', 'Topography.Code_C809', 'Topography.Code_other', 'Morphology.Description_ADENOCARCINOMA TUBULAR', 'Morphology.Description_ADENOCARCINOMA, SOE', 'Morphology.Description_CARCINOMA BASO CELULAR, SOE', 'Morphology.Description_CARCINOMA BASOCELULAR, SOE', 'Morphology.Description_CARCINOMA DE CELULAS ESCAMOSAS, SOE', 'Morphology.Description_CARCINOMA DUCTAL INFILTRANTE', 'Morphology.Description_CARCINOMA ESCAMOCELULAR, SOE', 'Morphology.Description_NEOPLASIA MALIGNA', 'Morphology.Description_NEOPLASMA MALIGNO', 'Morphology.Description_other', 'Code.of.Morphology_80003', 'Code.of.Morphology_80703', 'Code.of.Morphology_80903', 'Code.of.Morphology_81403', 'Code.of.Morphology_82113', 'Code.of.Morphology_85003', 'Code.of.Morphology_95903', 'Code.of.Morphology_97323', 'Code.of.Morphology_98613', 'Code.of.Morphology_other', 'Description.of.Disease_BRONQUIOS OU PULMOES, SOE', 'Description.of.Disease_COLO DO UTERO, SOE', 'Description.of.Disease_ESTOMAGO, SOE', 'Description.of.Disease_LOCALIZACAO PRIMARIA DESCONHECIDA', 'Description.of.Disease_MAMA , SOE', 'Description.of.Disease_PELE DE OUTRAS PARTES DA FACE', 'Description.of.Disease_PELE DO MEMBRO SUPERIOR E DO OMBRO', 'Description.of.Disease_PELE DO TRONCO', 'Description.of.Disease_PROSTATA', 'Description.of.Disease_other', 'Illness.Code_C169', 'Illness.Code_C349', 'Illness.Code_C443', 'Illness.Code_C445', 'Illness.Code_C446', 'Illness.Code_C509', 'Illness.Code_C61', 'Illness.Code_C80', 'Illness.Code_C859', 'Illness.Code_other', 'Diagnostic.means_CITOLOGIA', 'Diagnostic.means_CL?NICO', 'Diagnostic.means_HISTOLOGIA DO TUMOR PRIM?RIO', 'Diagnostic.means_PESQUISA', 'Diagnostic.means_SDO', 'Diagnostic.means_other', 'Extension_IN SITU', 'Extension_LOCALIZADO', 'Extension_MET?STASE', 'Extension_N?O SE APLICA', 'Type.of.Death_C?NCER', 'Type.of.Death_N?O C?NCER', 'Type.of.Death_other'])

columns = list(check.columns)
for i, c in enumerate(columns):
    if 'extension' in c.lower():
        print(f"{c} - {i}")

for i, c in enumerate(columns):
    if 'illness.code' in c.lower():
        print(f"{c} - {i}")

Extension_IN SITU - 133
Extension_LOCALIZADO - 134
Extension_MET?STASE - 135
Extension_N?O SE APLICA - 136
Illness.Code_C169 - 117
Illness.Code_C349 - 118
Illness.Code_C443 - 119
Illness.Code_C445 - 120
Illness.Code_C446 - 121
Illness.Code_C509 - 122
Illness.Code_C61 - 123
Illness.Code_C80 - 124
Illness.Code_C859 - 125
Illness.Code_other - 126


Data divided into MTL form

In [26]:
import numpy as np

# Take the maximum time and divide it into any number of intervals
Tmax = 7500
num_intervals = 7

intervals = [(i * (Tmax // num_intervals), (i + 1) * (Tmax // num_intervals)) for i in range(num_intervals)]

Y = np.zeros((len(time_all), num_intervals), dtype=np.int_)

# Until the event happens, value is 1. after that, it is 0
for i, time_val in enumerate(time_all):
    for j, (left, right) in enumerate(intervals):
        if time_val > right or (left < time_val <= right):
            Y[i, j] = 1

Y = torch.Tensor(Y)

print(Y.shape)

torch.Size([14755, 7])


In [27]:
# Creat mask matrix
W = np.zeros((len(time_all), num_intervals), dtype=np.int_)

# Until the time we have know he was alive the value is 1, after that is 0
for i, (time_val, event_val) in enumerate(zip(time_all, event_all)):
    for j, (left, right) in enumerate(intervals):
        if event_val == 0 and time_val < left:
            W[i, j] = 0
        else:
            W[i, j] = 1

W = torch.Tensor(W)

In [28]:
print(W.shape)
# So far all the codes are done by Xinyu

torch.Size([14755, 7])


## Define and Apply the DataLoader
- Introduced the dataloader to deal with the dataset, which would be helpful in the training and testing steps

In [29]:
# Jingyan adds the dataloader
from torch.utils.data import Dataset, DataLoader, random_split

class MultiTaskDataset(Dataset):
    def __init__(self, data, targets, masks, event_all):
        self.data = data
        self.targets = targets
        self.masks = masks
        self.event_all = event_all

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], [self.targets[i][idx] for i in range(len(self.targets))], [self.masks[i][idx] for i in range(len(self.masks))], self.event_all[idx]


In [30]:
for col in X.columns:       # modified by Ali Sam. (when I try to run the code, the transformation to tensor gives an error of data type, so I have converted the boolean coloumns to int dtype first)
    if X[col].dtype == bool:
        X[col] = X[col].astype(int)
X_use = torch.tensor(X.values, dtype=torch.float32)
X_use.shape

torch.Size([14755, 138])

In [31]:
# Convert n*m data into m * n * 1 
Y_transform = [Y[:, i:i+1] for i in range(Y.size(1))]
print(Y_transform[0].shape)
W_transform = [W[:, i:i+1] for i in range(W.size(1))]
print(W_transform[0].shape)

torch.Size([14755, 1])
torch.Size([14755, 1])


In [32]:
#Xinyu
full_dataset = MultiTaskDataset(X_use, Y_transform, W_transform, event_all)

train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size

train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True)

In [33]:
# created by Jingyan to check the data shape before training
for x, y, w, e in train_loader:
    print(x.shape)
    print(len(y), y[0].shape)
    print(len(w), w[0].shape)
    print(e.shape)
    in_features = x.shape[1]
    out_features = len(y)
    break

torch.Size([2048, 138])
7 torch.Size([2048, 1])
7 torch.Size([2048, 1])
torch.Size([2048])


In [34]:
out_features

7

In [35]:
for x, y, w, e in test_loader:
    print(x.shape)
    print(len(y), y[0].shape)
    print(len(w), w[0].shape)
    print(e.shape)
    break

torch.Size([2048, 138])
7 torch.Size([2048, 1])
7 torch.Size([2048, 1])
torch.Size([2048])


In [36]:
e_train = []
for x, y, w, e in train_loader:
  e_train.append(e)

event_train = torch.cat(e_train, dim=0)

In [37]:
event_train.shape

torch.Size([10240])

## Original Code (comment out)

In [38]:
# num_columns = Y_train.shape[1]
# Y_train_cols = []
# for i in range(num_columns):
#     col = Y_train[:, i]
#     var_name = 'Y' + str(i+1)
#     locals()[var_name] = col
#     Y_train_cols.append(locals()[var_name])

# num_test_columns = Y_test.shape[1]
# Y_test_cols = []
# for i in range(num_test_columns):
#     col = Y_test[:, i]
#     var_name = 'Y' + str(i+1)
#     locals()[var_name] = col
#     Y_test_cols.append(locals()[var_name])

In [39]:
# num_columns = W_train.shape[1]

# W_train_cols = []

# for i in range(num_columns):
#   col = W_train[:, i]
#   var_name = 'W' + str(i+1)
#   locals()[var_name] = col
#   W_train_cols.append(locals()[var_name])

# num_test_columns = W_test.shape[1]

# W_test_cols = []

# for i in range(num_test_columns):
#   col = W_test[:, i]
#   var_name = 'W' + str(i+1)
#   locals()[var_name] = col
#   W_test_cols.append(locals()[var_name])

In [40]:
# print(W_train_cols[1].shape)
# print(W_test_cols[1].shape)
# print(Y_train_cols[1].shape)
# print(Y_test_cols[1].shape)

## Model
- Changed the loss function to match the data format using dataloader

In [41]:
import torch.nn as nn
import torch.nn.functional as F

In [42]:
# Made by Xinyu and modified by Dr.Li and Jingyan
class MultiTaskModel(nn.Module):
    def __init__(self, in_features, out_features):
        super(MultiTaskModel, self).__init__()
        self.num_tasks = out_features
        self.input_features = in_features

        self.shared_layers = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.AlphaDropout(0.1),

            nn.Linear(128, 128),
            nn.ReLU(),
            nn.AlphaDropout(0.1),

            nn.Linear(128, 128),
            nn.ReLU(),
            nn.AlphaDropout(0.1)
        )
# A Classic Multi Task Learning Framework
        self.task_layers = nn.ModuleList([nn.Linear(128, 1) for _ in range(out_features)])#after your papaer I decided to use MTL to solve this prob

# This forward propagation logic defines the chain propagation of our idea
    def forward(self, x):
        shared_output = self.shared_layers(x)
        task_outputs = []
        for i, task_layer in enumerate(self.task_layers):
            if i == 0:
                task_output = torch.sigmoid(task_layer(shared_output))
            else:
                task_output = torch.sigmoid(task_layer(shared_output)) * task_outputs[-1]
            task_outputs.append(task_output)

        return task_outputs# Here I difined the S(x)


    # def custom_loss(self, task_outputs, Y_train_cols, W_train_cols):
    #     loss = 0
    #     for i, task_output in enumerate(task_outputs):
    #         task_target = Y_train_cols[i].unsqueeze(1)
    #         task_mask = W_train_cols[i].unsqueeze(1)
    #         task_loss = F.binary_cross_entropy_with_logits(task_output, task_target.float(), reduction='none')
    #         task_loss = task_loss * task_mask.float()
    #         loss += task_loss.sum() / task_mask.sum()
    #     return loss#There are how I use mask matrix W to control the loss function

# Modified by Jingyan
    def custom_loss(self, task_outputs, targets, masks):
        loss = 0
        for i, task_output in enumerate(task_outputs):
            task_target = targets[i]
            task_mask = masks[i]
            task_loss = F.binary_cross_entropy(task_output, task_target.float(), reduction='none')
            task_loss = task_loss * task_mask.float()
            loss += task_loss.sum() / task_mask.sum()
        return loss

## TASK 2

In [75]:
import torch
from tqdm import tqdm
import torch

#Ali Sam
import torch
from tqdm import tqdm
import torch


class SimplifiedAttributionPriorExplainer:
    def __init__(self, model):
        self.model = model

    def compute_gradients(self, input_tensor, interval):
        self.model.eval()
        input_tensor.requires_grad_(True)
        output = self.model(input_tensor)[interval].squeeze()
        output.backward(torch.ones_like(output))
        gradients = input_tensor.grad.data
        input_tensor.requires_grad_(False)

        return gradients.abs()  # Return the absolute value of the gradients to represent feature importance


E = 10 #150
T = 7  # Assuming there are 7 time intervals
lambda_smooth = 0.5  # Fixed regularization coefficient for Omega_smooth
loss_records = []
# Initialize model and optimizer
model = MultiTaskModel(in_features, out_features).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
explainer = SimplifiedAttributionPriorExplainer(model)


# Training Loop with Enhanced Dynamic Regularization
for e in range(1, E+1):
    total_loss = 0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {e}/{E}")
    for batch_idx, (X_train, targets, masks, event_train) in progress_bar:
        X_train = X_train.to(device)
        targets = [target.to(device) for target in targets]
        masks = [mask.to(device) for mask in masks]

        optimizer.zero_grad()
        task_outputs = model(X_train)
        # Base loss computation
        L_base = model.custom_loss(task_outputs, targets, masks)
        # Regularization terms calculation
        Rsmooth = 0  # Initialize smoothing regularization
        prev_attributions = None
        for i in range(T):
            current_attributions = explainer.compute_gradients(X_train, i)

            # Compute smoothing regularization
            if prev_attributions is not None:
                diff = current_attributions - prev_attributions
                norm_diff = torch.norm(diff, p=1)
                Rsmooth += norm_diff

            prev_attributions = current_attributions
        
        loss = L_base + (lambda_smooth * Rsmooth)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        progress_bar.set_postfix({'Loss': loss.item()})

    avg_loss = total_loss / len(train_loader)
    loss_records.append(avg_loss)
    print(f'End of Epoch {e}, Average Loss: {avg_loss:.4f}')

print('Training completed')












Epoch 1/10: 100%|██████████| 5/5 [00:00<00:00,  9.29it/s, Loss=204]


End of Epoch 1, Average Loss: 162.2230


Epoch 2/10: 100%|██████████| 5/5 [00:00<00:00,  8.46it/s, Loss=593]


End of Epoch 2, Average Loss: 405.4234


Epoch 3/10: 100%|██████████| 5/5 [00:00<00:00,  8.08it/s, Loss=788]


...

End of Epoch 8, Average Loss: 37.9142


Epoch 9/10: 100%|██████████| 5/5 [00:00<00:00,  8.25it/s, Loss=39.4]


End of Epoch 9, Average Loss: 39.4617


Epoch 10/10: 100%|██████████| 5/5 [00:00<00:00,  9.92it/s, Loss=40.2]

End of Epoch 10, Average Loss: 39.5350
Training completed


In [63]:
# Created by Jingyan
model.eval()
test_predictions = []

# Disable gradient calculations
with torch.no_grad():
    for X_test, _, _, _ in test_loader:
        # Forward pass
        X_test = X_test.to(device)
        task_outputs_ = model(X_test)

        # Store the predictions
        test_predictions.append(task_outputs_)

# Process the predictions as needed
# For example, converting them to a list or concatenating
# Here we concatenate the predictions for each task
test_predictions = [torch.cat([preds[i] for preds in test_predictions]) for i in range(len(test_predictions[0]))]

print(len(test_predictions))
print(test_predictions[0].shape)

7
torch.Size([2048, 1])


In [64]:
# Created by Jingyan
model.eval()
train_predictions = []

# Disable gradient calculations
with torch.no_grad():
    for X_train, _, _, _ in train_loader:
        # Forward pass
        X_train = X_train.to(device)
        task_outputs_ = model(X_train)

        # Store the predictions
        train_predictions.append(task_outputs_)

# Process the predictions as needed
# For example, converting them to a list or concatenating
# Here we concatenate the predictions for each task
train_predictions = [torch.cat([preds[i] for preds in train_predictions]) for i in range(len(train_predictions[0]))]

print(len(train_predictions))
print(train_predictions[0].shape)

7
torch.Size([10240, 1])


In [65]:
# made by Xinyu
def binarize_and_sum_columns(output_list):
    def binarize_list(input_list):
        tensor = torch.Tensor(input_list)
        # print(input_list.max() == input_list.min())
        binary_tensor = (tensor >= 0.5).float()
        return binary_tensor

    result = binarize_list(output_list[0])
    for i in range(1, len(output_list)):
        binary_column = binarize_list(output_list[i])
        print(binary_column.max() == binary_column.min())
        result += binary_column

    return result

# made by Xinyu
Y_hat_train = binarize_and_sum_columns(train_predictions)
Y_hat_train = Y_hat_train.squeeze()
print(Y_hat_train.shape)

Y_hat_test = binarize_and_sum_columns(test_predictions)
Y_hat_test = Y_hat_test.squeeze()
print(Y_hat_test.shape)


# created by Jingyan
# Get the true labels and status
train_trues = []
train_statuses = []
for _, train_targets, train_masks, train_status in train_loader:
    true_label = [train_targets[i]*train_masks[i] for i in range(len(train_targets))]
    train_trues.append(true_label)
    train_statuses.append(train_status)
train_trues = [torch.cat([preds[i] for preds in train_trues]) for i in range(len(train_trues[0]))]
train_statuses = torch.cat([status for status in train_statuses])

print(len(train_trues))
print(train_trues[0].shape)
print(train_statuses.shape)

test_trues = []
test_statuses = []
for _, test_targets, test_masks, test_status in test_loader:
    true_label = [test_targets[i]*test_masks[i] for i in range(len(test_targets))]
    test_trues.append(true_label)
    test_statuses.append(test_status)

test_trues = [torch.cat([preds[i] for preds in test_trues]) for i in range(len(test_trues[0]))]
test_statuses = torch.cat([status for status in test_statuses])

print(len(test_trues))
print(test_trues[0].shape)
print(test_statuses.shape)

# made by Xinyu
Y_true_train = binarize_and_sum_columns(train_trues)
Y_true_train = Y_true_train.squeeze()
print(Y_true_train.shape)

Y_true_test = binarize_and_sum_columns(test_trues)
Y_true_test = Y_true_test.squeeze()
print(Y_true_test.shape)

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
torch.Size([10240])
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
torch.Size([2048])
7
torch.Size([10240, 1])
torch.Size([10240])
7
torch.Size([2048, 1])
torch.Size([2048])
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
torch.Size([10240])
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
torch.Size([2048])


In [66]:
from tqdm import trange
class Cindex(torch.nn.Module):
    def __init__(self):
        super(Cindex, self).__init__()

    def forward(self, y, y_hat, status):
        if not torch.is_tensor(y):
            y = torch.Tensor(y)
        if not torch.is_tensor(y_hat):
            y_hat = torch.Tensor(y_hat)
        if not torch.is_tensor(status):
          status = torch.Tensor(status)

        N = y.size(0)
        total_pairs = 0
        c = 0

        for i in trange(N):
            for j in range(i + 1, N):
                a = y[i]
                b = y[j]
                a_hat = y_hat[i]
                b_hat = y_hat[j]
                astatus = status[i]
                bstatus = status[j]
                if (a >= b and a_hat >= b_hat and bstatus == 1) or (a <= b and a_hat <= b_hat and astatus == 1):
                    c += 1
                if (a <= b and astatus==1) or (b <= a and bstatus == 1):
                    total_pairs += 1

        outcome = c / total_pairs
        return outcome

cindex_calculator = Cindex()
# c11_train = cindex_calculator(Y_true_train, Y_hat_train, train_statuses)
# print(c11_train)
c11_test = cindex_calculator(Y_true_test, Y_hat_test, test_statuses)
print(c11_test)

  1%|▏         | 30/2048 [00:05<06:06,  5.50it/s]


KeyboardInterrupt: 

In [67]:
Y_true_test

tensor([4., 0., 3.,  ..., 1., 3., 2.])

In [69]:
Y_hat_test.max()

tensor(0., device='cuda:0')